In [1]:
## Based on http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#tutorial-setup

from sklearn.datasets import fetch_20newsgroups

In [2]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

In [3]:
from sklearn.datasets import fetch_20newsgroups

In [13]:
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

In [14]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [15]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [16]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
count_vect = CountVectorizer()

In [19]:
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [20]:
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [21]:
count_vect.vocabulary_.get(u'algorithm')

4690